In [ ]:
#############
# IMPORT 
#
import numpy as np
import pickle
import matplotlib.pyplot as plt
import pathlib
import time

from naludaq.board import Board, startup_board
from naludaq.controllers import DACController, PedestalsController, BoardController
from naludaq.controllers.pedestals_controller import AAv3PedestalsController
#from nalu_analyzer.data_collector import DataCollector
from naludaq.communication import (
    DigitalRegisters,
    ControlRegisters,
    AnalogRegisters,
)
#from naludaq.daq.workers import Parser

from naludaq.controllers.si5341_controller import Si5341_Controller

#from nalu_analyzer import plotter

%load_ext autoreload
%autoreload 2

import naludaq
naludaq.__version__

from logging import getLogger, Formatter, StreamHandler, INFO, DEBUG, ERROR

def setup_logger(level=DEBUG):
    logger = getLogger()
    handler = StreamHandler()
    handler.setFormatter(Formatter('%(asctime)s %(name)-30s [%(levelname)-6s]: %(message)s'))
    logger.addHandler(handler)
    logger.setLevel(level)
    uart = getLogger('naludaq.board.connections._UART')
    uart.setLevel(ERROR)
    
    return logger

try:
    logger.info("logger already setup")
except:
    logger = setup_logger()
    
uart = getLogger('naludaq.board.connections._UART')
uart.setLevel(DEBUG)
    
working_dir = pathlib.Path().cwd()    
    

In [ ]:
from naludaq.board import Board, startup_board

varc = Board('aardvarcv3')

# FMC Nexys Video
varc.get_uart_connection(comport='COM16', baud=115200)

# HIPeR BOARD PARTIAL BUILD
#varc.get_uart_connection(comport='COM18', baud=115200)

# HIPeR BOARD FULL BUILD
#varc.get_uart_connection(comport='COM20', baud=115200)


#varc.load_registers(working_dir / 'configs' / 'aardvarcv3_params.yml')
varc.load_registers(working_dir / 'configs' / 'aardvarcv3_HIPeR.yml')

# FMC CARD
varc.load_clockfile(working_dir / 'configs' / 'Si5341-RevD-HIPeR-NexysFMC_78_125SysClk_312_5GccClk-Registers.txt')

# HIPeR CARD
#varc.load_clockfile(working_dir / 'configs' / 'Si5341-RevD-HIPeR_78_125SysClk_312_5GccClk-Registers.txt')


################################
# CONNECT TO THE FMC CARD
varc.connect()

################################
# UPDATE FPGA AND ASIC REGISTERS
# IN THE PARAMS FILE BE
# SURE RxOut IS ENABLED ON ALL
# ASICS OTHERWISE THEY WILL NOT 
# GET CONFIGUREDED
# REGISTER 25hex SET TO ALL 1'S
# ControlRegisters(varc).write('RxOut_en',65535);
#  RxOut_en:
#  - 65535
#  - 37
#  - 0
#  - 16 
#

# RXOut Enable ON
#varc.connection.send("AF25FFFF")
#time.sleep(0.5)

startup_board(varc);


In [ ]:
# RESET AARDVARC SERIAL INTERFACE
# DO NOT RESET THE INTERFACE UNLESS YOU CAN REALOAD THE ASIC REGISTERS
# If you use SysRst or RegClr the ASIC register space is cleared to zero
# you must reload the ASIC registers space
# To reset turn off the serial Rx and Tx and reset AARDVARC.
# before asserting sysclr and regclr
#
varc.connect()

# TXIn Enable OFF
ControlRegisters(varc).write('txin_en',0);
time.sleep(0.5)

# RXOut Enable OFF
ControlRegisters(varc).write('rxout_en',0);
time.sleep(0.5)

# RESET AARDVARC REGISTERS AND SYSTEM
ControlRegisters(varc).write('regclr',1);
time.sleep(0.5);
ControlRegisters(varc).write('regclr',0);
time.sleep(0.5);
ControlRegisters(varc).write('sysrst',1);
time.sleep(0.5);
ControlRegisters(varc).write('sysrst',0);
time.sleep(0.5);

# LET AARDVARC SETTLE
time.sleep(1.0);



In [ ]:
# SET POLARITY TO ASIC RXOUT
# IF POLAROITY AND BAUD IS NOT CORRECT BUSY_LOCKED WILL BE 0
# IF POLARITY AND BAUD ARE CORRECT BUSY_LOCKED WILL BE 1
ControlRegisters(varc).write('rxout_pol',0);
ControlRegisters(varc).read('busy_locked');

In [ ]:
# SET POLARITY TO ASIC TXIN
# IF POLAROITY AND BAUD IS NOT CORRECT IDLE_DETECT WILL BE 0
# IF POLARITY AND BAUD ARE CORRECT IDLE_DETETCT WILL BE 1
ControlRegisters(varc).write('txin_pol',0);
ControlRegisters(varc).read('idle_det');

In [ ]:
# FMC Nexys Video Card DAC I2C interface
#naludaq.controllers.external_dac()
DACController(varc).set_dacs(1000, [0])
DACController(varc).set_dacs(1500, [1])
DACController(varc).set_dacs(2000, [2])
DACController(varc).set_dacs(2500, [3])

In [ ]:
# HIPeR Board SPI DAC Interface LEFT SIDE ASIC 0 
# Chip Id is 0-13
# Addr 15 all channels
# Cmd 3 write, update, and enable selected channels
varc.connect();
ControlRegisters(varc).write('dac_l_value',2048);
ControlRegisters(varc).write('dac_l_asic',6);
ControlRegisters(varc).write('dac_l_addr',15);
ControlRegisters(varc).write('dac_l_cmd',3);
ControlRegisters(varc).write('dac_l_update',0);
ControlRegisters(varc).write('dac_l_update',1);
ControlRegisters(varc).write('dac_l_update',0);

In [ ]:
# HIPeR Board SPI DAC Interface RIGHT SIDE ASIC 0 
# Chip Id is 0-13
# Chip Id is 0-13
# Addr 15 all channels
# Cmd 3 write, update, and enable selected channels
varc.connect();
ControlRegisters(varc).write('dac_r_value',2048);
ControlRegisters(varc).write('dac_r_asic',0);
ControlRegisters(varc).write('dac_r_addr',15);
ControlRegisters(varc).write('dac_r_cmd',3);
ControlRegisters(varc).write('dac_r_update',0);
ControlRegisters(varc).write('dac_r_update',1);
ControlRegisters(varc).write('dac_r_update',0);

In [ ]:
# TXIn Enable OFF
ControlRegisters(varc).write('txin_en',0);
time.sleep(0.5)

# RXOut Enable OFF
ControlRegisters(varc).write('rxout_en',0);
time.sleep(0.5)

In [ ]:
#
# Configufure for Acquire
# 16 WINDOWS
# START 0
# END 15
# Set Acquire Mode : Trigger=External : Acquisition=Forced  : Pedestal=Rawdata
#

varc.connect()

#####################################
# BE SURE RXOut Enable IS ON
ControlRegisters(varc).write('rxout_en',65535);
time.sleep(0.1)

#####################################
# TXIn Enable OFF TO AVOID UNWANTED RETURN DATA.
# CAN BE ON IF YOU WANT RETURN DATA FOR EACH COMMAND
ControlRegisters(varc).write('txin_en',0);
time.sleep(0.1)

#####################################
# The Following registers are set during
# startup_board(varc) but can be written
# again
#
# MISCREG
DigitalRegisters(varc).write('miscreg',2);
time.sleep(0.1) 

# PLLMISC
DigitalRegisters(varc).write('pllmisc',0);
time.sleep(0.1) 

# TXSPEED
DigitalRegisters(varc).write('txspeed',2);
time.sleep(0.1) 

# WAITREAD
DigitalRegisters(varc).write('waitread',1);
time.sleep(0.1)

#######################################
# Configure for Acquire
######################################
# wraddrstart
DigitalRegisters(varc).write('wraddrstart',0);
time.sleep(0.1) 

# writeAddressStop
DigitalRegisters(varc).write('wraddrstop',127);
time.sleep(0.1)

################################################
# Set Acquire Mode : Trigger=External : Acquisition=Forced  : Pedestal=Rawdata
# Cmd = 0XB0300010;
varc.connection.send("B0300010")
time.sleep(0.1)

In [ ]:
######################################
# TXIn Enable ON to enable return data capture
time.sleep(0.1)
ControlRegisters(varc).write('txin_en',1);

# TRIGGER
ControlRegisters(varc).write('exttrig',1);
ControlRegisters(varc).write('exttrig',0);

In [ ]:
# SET TRIGGER MASK
# soon to be trig_mask
varc.connect()
ControlRegisters(varc).write('exttrig_mask',1);

In [ ]:
varc.disconnect()

In [ ]:
varc.connect()